# 1. pdf 로더

In [1]:
file_path = 'file/현행법령/도로교통법(법률)(제19745호)(20241025).pdf'

## 1-1) PyPDFLoader 사용 예제

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(file_path)
pages = loader.load_and_split()

print(f"문서의 수: {len(pages)}")
print(f"\n[페이지내용]\n{pages[0].page_content[:500]}")
print(f"\n[metadata]\n{pages[0].metadata}\n")

문서의 수: 63

[페이지내용]
법제처                                                            1                                                       국가법령정보센터
도로교통법 
 
도로교통법 
[시행 2024. 10. 25.] [법률 제19745호, 2023. 10. 24., 일부개정] 
경찰청 (교통기획계(법제총괄, 안전표지, 보호구역)) 02-3150-2251 
경찰청 (교통안전계(안전, 단속, 어린이통학버스)) 02-3150-2252 
경찰청 (운전면허계(운전면허)) 02-3150-2253 
경찰청 (첨단교통계(신호, 무인단속장비)) 02-3150-2851 
경찰청 (교통조사계(교통사고조사)) 02-3150-2552 
       제1장 총칙
 
제1조(목적) 이 법은 도로에서 일어나는 교통상의 모든 위험과 장해를 방지하고 제거하여 안전하고 원활한 교통을 확보
함을 목적으로 한다.
 
제2조(정의) 이 법에서 사용

[metadata]
{'source': 'file/현행법령/도로교통법(법률)(제19745호)(20241025).pdf', 'page': 0}



## 1-2) PyMuPDFLoader 사용 예제

In [3]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(file_path)
pages = loader.load()

print(f"문서의 수: {len(pages)}")
print(f"\n[페이지내용]\n{pages[0].page_content[:500]}")
print(f"\n[metadata]\n{pages[0].metadata}\n")

문서의 수: 63

[페이지내용]
법제처                                                            1                                                       국가법령정보센터
도로교통법
 
도로교통법
[시행 2024. 10. 25.] [법률 제19745호, 2023. 10. 24., 일부개정]
경찰청 (교통기획계(법제총괄, 안전표지, 보호구역)) 02-3150-2251
경찰청 (교통안전계(안전, 단속, 어린이통학버스)) 02-3150-2252
경찰청 (운전면허계(운전면허)) 02-3150-2253
경찰청 (첨단교통계(신호, 무인단속장비)) 02-3150-2851
경찰청 (교통조사계(교통사고조사)) 02-3150-2552
       제1장 총칙
 
제1조(목적) 이 법은 도로에서 일어나는 교통상의 모든 위험과 장해를 방지하고 제거하여 안전하고 원활한 교통을 확보
함을 목적으로 한다.
 
제2조(정의) 이 법에서 사용하는 용어의 뜻

[metadata]
{'source': 'file/현행법령/도로교통법(법률)(제19745호)(20241025).pdf', 'file_path': 'file/현행법령/도로교통법(법률)(제19745호)(20241025).pdf', 'page': 0, 'total_pages': 63, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'iText 2.1.7 by 1T3XT', 'creationDate': "D:20240906151240+09'00'", 'modDate': "D:20240906151240+09'00'", 'trapped': ''}



## 1-3) PyPDFDirectoryLoader 사용 예제

In [7]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

pdf_dir = "file/도로 교통법/"

loader = PyPDFDirectoryLoader(pdf_dir)
docs = loader.load()

print(f"로드된 문서의 수: {len(docs)}")
for doc in docs[:2]:  # 처음 2개 문서만 출력
    print(f"\n[문서내용]\n{doc.page_content[:200]}")
    print(f"\n[metadata]\n{doc.metadata}\n")

로드된 문서의 수: 456

[문서내용]
법제처                                                            1                                                       국가법령정보센터
도로교통법 
 
도로교통법 
[시행 2025. 3. 20.] [법률 제20375호, 2024. 3. 19., 일부개정] 
경찰청 

[metadata]
{'source': 'file/도로 교통법/도로교통법(법률)(제20375호)(20250320).pdf', 'page': 0}


[문서내용]
법제처                                                            2                                                       국가법령정보센터
도로교통법 
13의2. “회전교차로”란 교차로 중 차마가 원형의 교통섬(차마의 안전하고 원활한 교통처리나 보행자 도로횡단의 안
전

[metadata]
{'source': 'file/도로 교통법/도로교통법(법률)(제20375호)(20250320).pdf', 'page': 1}



# 2. 전처리

In [10]:
from langchain_community.document_loaders import PyMuPDFLoader

def read_pdf(file_path):
    loader = PyMuPDFLoader(file_path)
    return loader.load()

In [16]:
import re

def clean_text(text):
    text = re.sub(r"법제처\s+.*?\s+국가법령정보센터", "", text)

    # 괄호 안의 값 제거: <>, [], (), 그리고 전화번호 형식 제거
    text = re.sub(r"<.*?>|\[.*?]|\(.*?\)", "", text, flags=re.DOTALL)
    
    # 괄호와 전화번호 같이 있는 경우 제거 (ex: (내용) 02-1234-5678 또는 (내용) 010-123-4567)
    text = re.sub(r"\(.*?\)\s*\d{2,4}[-.\s]\d{3,4}[-.\s]\d{4}", "", text)
    
    # 일반 전화번호 형식 제거 (괄호와 상관없이 나타나는 번호도 제거)
    text = re.sub(r"\d{2,4}[-.\s]\d{3,4}[-.\s]\d{4}", "", text)

    # 여분의 공백을 하나의 공백으로 대체
    text = re.sub(r"\s+", " ", text)

    return text.strip()

In [12]:
file_path = 'file/현행법령/도로교통법(법률)(제19745호)(20241025).pdf'
documents = read_pdf(file_path)

### 전처리 전

In [18]:
print(documents[0].page_content)

법제처                                                            1                                                       국가법령정보센터
도로교통법
 
도로교통법
[시행 2024. 10. 25.] [법률 제19745호, 2023. 10. 24., 일부개정]
경찰청 (교통기획계(법제총괄, 안전표지, 보호구역)) 02-3150-2251
경찰청 (교통안전계(안전, 단속, 어린이통학버스)) 02-3150-2252
경찰청 (운전면허계(운전면허)) 02-3150-2253
경찰청 (첨단교통계(신호, 무인단속장비)) 02-3150-2851
경찰청 (교통조사계(교통사고조사)) 02-3150-2552
       제1장 총칙
 
제1조(목적) 이 법은 도로에서 일어나는 교통상의 모든 위험과 장해를 방지하고 제거하여 안전하고 원활한 교통을 확보
함을 목적으로 한다.
 
제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2012. 3. 21., 2013. 3. 23., 2014. 1. 28., 2014. 11. 19.,
2017. 3. 21., 2017. 7. 26., 2017. 10. 24., 2018. 3. 27., 2020. 5. 26., 2020. 6. 9., 2020. 12. 22., 2021. 10. 19., 2022. 1.
11., 2023. 4. 18., 2023. 10. 24.>
1. “도로”란 다음 각 목에 해당하는 곳을 말한다.
가. 「도로법」에 따른 도로
나. 「유료도로법」에 따른 유료도로
다. 「농어촌도로 정비법」에 따른 농어촌도로
라. 그 밖에 현실적으로 불특정 다수의 사람 또는 차마(車馬)가 통행할 수 있도록 공개된 장소로서 안전하고 원활
한 교통을 확보할 필요가 있는 장소
2. “자동차전용도로”란 자동차만 다닐 수 있도록 설치된 도로를 말한다.
3. “고속도로”란 자동차의 고속 운행에만 사용하기 위하여 지정된 도로를 말한다.
4. “차도”(車道)

### 전처리 후

In [19]:
print(clean_text(documents[0].page_content))

도로교통법 도로교통법 경찰청 ) 경찰청 ) 경찰청 ) 경찰청 ) 경찰청 ) 제1장 총칙 제1조 이 법은 도로에서 일어나는 교통상의 모든 위험과 장해를 방지하고 제거하여 안전하고 원활한 교통을 확보 함을 목적으로 한다. 제2조 이 법에서 사용하는 용어의 뜻은 다음과 같다. 1. “도로”란 다음 각 목에 해당하는 곳을 말한다. 가. 「도로법」에 따른 도로 나. 「유료도로법」에 따른 유료도로 다. 「농어촌도로 정비법」에 따른 농어촌도로 라. 그 밖에 현실적으로 불특정 다수의 사람 또는 차마가 통행할 수 있도록 공개된 장소로서 안전하고 원활 한 교통을 확보할 필요가 있는 장소 2. “자동차전용도로”란 자동차만 다닐 수 있도록 설치된 도로를 말한다. 3. “고속도로”란 자동차의 고속 운행에만 사용하기 위하여 지정된 도로를 말한다. 4. “차도”란 연석선, 안전표지 또는 그와 비 슷한 인공구조물을 이용하여 경계를 표시하여 모든 차가 통행할 수 있도록 설치된 도로의 부분을 말한다. 5. “중앙선”이란 차마의 통행 방향을 명확하게 구분하기 위하여 도로에 황색 실선이나 황색 점선 등의 안전표 지로 표시한 선 또는 중앙분리대나 울타리 등으로 설치한 시설물을 말한다. 다만, 제14조제1항 후단에 따라 가변 차로가 설치된 경우에는 신호기가 지시하는 진행방향의 가장 왼쪽에 있는 황색 점선을 말한다. 6. “차로”란 차마가 한 줄로 도로의 정하여진 부분을 통행하도록 차선으로 구분한 차도의 부분을 말한다. 7. “차선”이란 차로와 차로를 구분하기 위하여 그 경계지점을 안전표지로 표시한 선을 말한다. 7의2. “노면전차 전용로”란 도로에서 궤도를 설치하고, 안전표지 또는 인공구조물로 경계를 표시하여 설치한 「도시 철도법」 제18조의2제1항 각 호에 따른 도로 또는 차로를 말한다. 8. “자전거도로”란 안전표지, 위험방지용 울타리나 그와 비슷한 인공구조물로 경계를 표시하여 자전거 및 개인형 이 동장치가 통행할 수 있도록 설치된 「자전거 이용 활성화에 관한 법률」 제3조 각 호의 도로를 말한